<a href="https://colab.research.google.com/github/eznrr/projeto-sentiment140/blob/main/Pre_processamento_sentiment140.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

> Importando o DataSet

[Link para Download do DataSet.](https://www.kaggle.com/datasets/kazanova/sentiment140)

In [ ]:
from google import colab
colab.drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/projeto-oficina-sentiment140/Dataset/training.1600000.processed.noemoticon.csv', encoding='latin-1')

# Pré-Processamemto

**Etapas:**
- 1 - Visualização
- 2 - Renomeação de Colunas
- 3 - Imputação(Valores Faltando)
- 4 - Processamento de Linguagem Natural
- 5 - Normalização/Padronização

## Visualização

In [ ]:
# Informações gerais sobre o DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599999 entries, 0 to 1599998
Data columns (total 6 columns):
 #   Column                                                                                                               Non-Null Count    Dtype 
---  ------                                                                                                               --------------    ----- 
 0   0                                                                                                                    1599999 non-null  int64 
 1   1467810369                                                                                                           1599999 non-null  int64 
 2   Mon Apr 06 22:19:45 PDT 2009                                                                                         1599999 non-null  object
 3   NO_QUERY                                                                                                             1599999 non-null  object
 4   _

In [ ]:
# Inicio do DataFrame
df.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
df.shape

(1599999, 6)

## Renomeando as Colunas

In [ ]:
# Renomear as colunas para facilitar a manipulação
df.columns = ['target', 'id', 'date', 'flag', 'user', 'text']

print("Colunas do dataset:", df.columns)

Colunas do dataset: Index(['target', 'id', 'date', 'flag', 'user', 'text'], dtype='object')


In [ ]:
df.head()

,target,id,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


## Imputação

Imputação consiste na técninca de lidar com valores vazios, seja removendo suas linhas, ou as substituindo pela média, moda ou mediana.

Decisões em cima dos dados vazios:
- **Média**
  - Desvio Padrão
- **Moda**
- **Mediana**
- **Remoção Total:** Remoção de todos os valores faltantes, importante para colunas inuteis.

Funções usadas:
- `.fillna()` : preenche os valores vazios com algum valor.
    - `.fillna(df['exemplo'].mean(), inplace=True)`
- `.replace()` : substitui um valor por outro.
    - `df['floor'].replace('-', np.nan)`
- `.dropna()` : remove linhas que contem valores vazios.
    - `dropna(subset=['exemplo'], inplace=True)`
- `drop()` : remove uma coluna.
    - `.drop(columns=['exemplo_1', 'exemplo_2', ..., 'exemplo_n'], inplace=True)`

obs: o uso do `inplace=True`, faz com que os dados alterados, sejam alterados no DataFrame príncipal.

In [ ]:
# Verificar valores ausentes
df.isnull().sum()

,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [ ]:
df['target'].value_counts()

,count
target,
4,800000
0,799999


> Como todas as linhas do DatFrame estão categorizadas como positivo e negativo, confirmamos a não existência de valores vazios.

In [ ]:
# Decisão de remover a coluna 'flag'
df.drop(columns=['flag'], inplace=True)

## Lidando com Textos

> Diferente do DataSet de casas, aqui estamos lidando com **textos complexos**, que contém **informação relevantes** para prever emoções.

> A partir disso, aplicamos tecnicas presentes no **Processamento de Linguagem Natural**, que basicamente, permite o computador entender e interpretar a lingua humana.

### Remoção de caracteres indevidos

Explicação da biblioteca Re: É usada para trabalhar com expressões regulares, que são sequências de caracteres que formam um padrão de pesquisa. Elas são extremamente poderosas para manipulação de strings, como busca, substituição e extração de padrões em textos.

Comandos usados:


```
re.sub
```

Substitui um conjunto de dados de uma string por outro.


```
text = re.sub(r'@\w+', '', text) substitui o @ e alfanuméricos(\w) +, por espaço em branco
```

Metacaracteres: Símbolos com significados especiais.
- `[ ]`: Definindo um conjunto de caracteres. Ex: [abc] corresponde a a, b ou c.
- `^`: Indica o início da string.
- `+`: Informa a substituição de uma ou mais ocorrências.

Sequências de Escape:
- `\w` : Qualquer caractere alfanumérico [a-zA-Z0-9_].
- `\s` : Qualquer espaço em branco.


*OBS: não esquecer de usar `r` no começo da função `.sub()` , pois ela impede erros associados a quebra de linha da formatação padrão do python.*

In [ ]:
df.head()

,target,id,date,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,joy_wolf,@Kwesidei not the whole crew


In [ ]:
# biblioteca para remapear texto
import re

# Função para remover URLs
def remove_urls(text):
    return re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

# Função para remover menções, hashtags, emojis e caracteres especiais
def clean_text(text):
    text = re.sub(r'@\w+', '', text)  # Remove menções

    # Agora, remova as Hashtags
    text = re.sub(r'#', '', text)

    text = re.sub(r'RT[\s]+', '', text)  # Remove RT
    text = re.sub(r'[^\w\s]', '', text)  # Remove todos os caracteres especiais
    text = text.encode('ascii', 'ignore').decode('ascii')  # Remove emojis

    return text

In [ ]:
%%time

# Aplicar as funções de limpeza
df['text'] = df['text'].apply(remove_urls)

# Use o apply para a função de limpar texto
df['text'] = df['text'].apply(clean_text)

CPU times: user 12.8 s, sys: 173 ms, total: 13 s
Wall time: 13.5 s


In [ ]:
# Imprima as primeiras colunas do DataSet
df.head()

,target,id,date,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,scotthamilton,is upset that he cant update his Facebook by t...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,mattycus,I dived many times for the ball Managed to sa...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,Karoli,no its not behaving at all im mad why am i he...
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,joy_wolf,not the whole crew


### Tokenizar o texto e remover stop words

Explicação NLTK: É uma biblioteca usada para tratamento de texto para modelagem de dados.

Tokenização:
- É o processo de dividir um texto em unidades menores, como palavras ou frases.


```
tokens = word_tokenize(text.lower())
```



Stopwords:
- São palavras comuns (como "a", "e", "o", "em") que geralmente são removidas de um texto durante o processamento, pois não carregam muito significado semântico.


```
stop_words = set(stopwords.words('english'))
```


```
filtered_tokens = [word for word in tokens if word not in stop_words and word.isalnum()]
```




In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
# Baixar os recursos necessários do NLTK
nltk.download('punkt')
nltk.download('stopwords')

# Definir as stop words em inglês
stop_words = set(stopwords.words('english'))

# Função para tokenizar e remover stop words
def preprocess_text(text):
    tokens = word_tokenize(text.lower())  # Tokenizar e converter para minúsculas
    filtred_tokens = []

    for word in tokens:
        if word not in stop_words and word.isalnum():
            filtred_tokens.append(word)

    return " ".join(filtred_tokens)  # Unir os tokens em uma string novamente

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
%%time

# aplique esse função que acabamos de desenvolver, criqando uma nova coluna
df['clean_text'] = df['text'].apply(preprocess_text)

CPU times: user 2min 56s, sys: 592 ms, total: 2min 57s
Wall time: 3min 2s


In [ ]:
# Imprima a cabeça do DataFrame final
df.head()

,target,id,date,user,text,clean_text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,scotthamilton,is upset that he cant update his Facebook by t...,upset cant update facebook texting might cry r...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,mattycus,I dived many times for the ball Managed to sa...,dived many times ball managed save 50 rest go ...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,ElleCTF,my whole body feels itchy and like its on fire,whole body feels itchy like fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,Karoli,no its not behaving at all im mad why am i he...,behaving im mad cant see
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,joy_wolf,not the whole crew,whole crew


# Exportando o DataFrame

> Com o DataFrame limpo, agora podemos exportar o mesmo para as etapas de visualização e modelagem.

In [ ]:
df.to_csv('sentiment_limpo.csv', index=False)